In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:


from core.store import FeatureStore
from SpectraDataset import SpectraDataset
import polars as pl
import numpy as np
from index.frame import IndexFrame
from processing import ProcessingRegistry

# ── fake minimal dataset ─────────────────────────────────────────────────────
index = pl.DataFrame(
    {
        "row": [0, 1],
        "sample": [0, 1],
        "origin": [0, 0],
        "partition": ["train","train"],
        "group": ["A","A"],
        "branch": ["main","main"],
        "processing": [100, 100],
    }
)
ds = SpectraDataset(
    index=IndexFrame(index),
    store=FeatureStore(),
    processing=ProcessingRegistry(),
)
# add one block for proc 100
src_id = ds.add_block(np.zeros((2, 3), np.float32), processing_id=100, n_users=2)

assert ds.store._refcounts[100] == 2

# duplicate one row twice → +2 refs
ds.duplicate_rows([0], 2)
assert ds.store._refcounts[100] == 4

# move three rows to proc 200 → 100 loses 3 refs, 200 gains 3
ds.set_processing(pl.col("row") < 3, new_processing_id=200)
assert ds.store._refcounts[100] == 1
assert ds.store._refcounts[200] == 3


TypeError: SpectraDataset.__init__() got an unexpected keyword argument 'index'